<a href="https://colab.research.google.com/github/parvathysarat/kg-qa/blob/master/qa_task_metaqa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Size of our transG embeddings : 50

In [0]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import os
import time, datetime
import sys
import json
import os

In [1]:
!git clone https://github.com/parvathysarat/kg-qa
# ./kg-qa/data/transg/ has the embeddings obtained by TransG for MetaQA dataset

Cloning into 'kg-qa'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 46 (delta 16), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [0]:
os.chdir('./kg-qa/data/transg/')

In [0]:
import glob
import shutil
for file in glob.glob('./kg-qa/data/*.txt'):
  shutil.move(file,'./kg-qa/data/transg/')

In [0]:
# entities.txt from MetaQa : list of entities

def get_entities():
  with open('../MetaQA/entities.txt') as f:
    entities = {id:line[:-1] for id,line in enumerate(f)}
    print(len(entities),entities[0])
    return entities
entities = get_entities()

In [0]:
# entity.txt storing TransG embeddings of MetaQA

def get_num_entity(param='num'):
   with open('entity.txt') as f:
    if param=='num':
      return sum([1 for line in f])
    if param=='weights':
      arr = []
      weights_dict = {}
      ct=0
      for line in f:
        entity = entities[ct]
        ct+=1
        for idx,el in enumerate(line.split()):
          # numeric entities are not vectorized
          if el.isnumeric() and idx==0: 
            continue

          if (' '.join(line.split()[:idx]))==entity:
            arr.append(np.array(line.split()[idx:],dtype=np.float32))
            weights_dict[entity] = np.array(line.split()[idx:],dtype=np.float32)
            break
      print(len(weights_dict),ct)
    pretrained_weights = torch.FloatTensor(arr)  
    return weights_dict, pretrained_weights
num_entities = get_num_entity()



In [102]:
num_entities = get_num_entity()
weights_dict, pretrained_weights = get_num_entity('weights')

43232 43234


In [0]:

def initialize_embeddings():
  entity_embeddings = nn.Embedding.from_pretrained(pretrained_weights)  
  # entity_embeddings = nn.Embedding(num_embeddings=num_entities+1, embedding_dim=50,padding_idx=num_entities)
  # entity_embeddings.weight = nn.Parameter(get_num_entity('weights'))
  entity_embeddings.weight.requires_grad = False  
  return entity_embeddings
entity_embeddings = initialize_embeddings()

In [110]:
entity_embeddings

Embedding(43232, 50)